In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tire
import car
from scipy.optimize import newton
con = 180/np.pi

In [2]:
params = {
    'W_uf' : 111,    #Front unsprung weight (N)
    'W_ur' : 111,    #Rear unsprung weight  (N)
    'W_s'  : 2446,   #Sprung weight         (N)
    # CG locations
    'z_wf' : 0.21,   #Front unsprung height (m)
    'z_wr' : 0.21,   #Rear unsprung height  (m)
    'h_s'  : 0.23,   #Sprung height         (m)
    'a_s'  : 0.76,   #Sprung distance from front axle (m)
    # Car geometry
    'l'    : 1.524,  #Wheelbase   (m)
    't_f'  : 1.22,   #Front track (m)
    't_r'  : 1.22,   #Rear track  (m)
    # Roll center heights
    'z_rf' : 0.038,  #Front roll center height (m)
    'z_rr' : 0.076,  #Rear roll center height  (m)
    # Roll rates
    'K_f'  : 24.5e3, #Front roll rate (N-m/rad)
    'K_r'  : 24.5e3, #Rear roll rate  (N-m/rad)
    # Steering
    'ack'  : 0.5,    #Steering Ackerman (%)
    'toe_f': 0.5,   #Front toe (deg)
    'toe_r': 1.0,    #Rear toe  (deg)
    'tire' : tire.Hoosier10X7()
}
C = car.Car(params)

Roll axis inclination: 1.43 deg
Unsprung CG height from roll axis: 0.17 m
Roll sensitivity: -0.50 deg/g
Front lateral weight transfer: 230.72 N/g
Rear lateral weight transfer:  268.52 N/g
Longitudinal weight transfer:       399.74 N/g
Front static wheel load:            -668.60 N
Rear static wheel load:             -665.40 N
Total weight:     -2668.00 N


In [ ]:
N = 50
r = 8
beta = np.linspace(-2.5, 8, N)
beta *= 1/con
delta = np.zeros(N, dtype='double')
Ay = np.zeros(N, dtype='double')
Fx = np.zeros(N, dtype='double')
load = np.zeros(N, dtype=[('fr', 'double'), ('fl', 'double'), ('rr', 'double'), ('rl', 'double')])
KA = np.zeros(N, dtype=[('fr', 'double'), ('fl', 'double'), ('rr', 'double'), ('rl', 'double')])
SA = np.zeros(N, dtype=[('fr', 'double'), ('fl', 'double'), ('rr', 'double'), ('rl', 'double')])
F = {}
FE = {}
for i in range(N):
    delta[i] = C.get_delta(r, beta[i])
    Ay[i], Fx[i], load[i], KA[i], SA[i], F[i], FE[i] = C.Ay_Fx_from_angle(r, beta[i], delta[i], extra=True)

sel = Ay > 0
plt.plot(beta[sel]*con, delta[sel]*con, 'tab:blue')
plt.title('Lateral acceleration and steer angle')
plt.xlabel(r'$\beta$ (deg)')
plt.ylabel(r'$\delta$ (deg)')
plt.twinx()
plt.plot(beta[sel]*con, Ay[sel], 'tab:red')
plt.ylabel(r'Ay (g)')
plt.legend([r'Ay'])
plt.show()

In [ ]:
# First step, analyze the existing car design to see where improvement can be made
sel = np.invert(np.isnan(Ay))
betaM = beta[sel][0]
deltaM = delta[sel][0]
AyM, FxM, loadM, KAM, SAM, FM, FEM = C.Ay_Fx_from_angle(r, betaM, deltaM, extra=True)
SAP = C.calc_peak_slip_angles(loadM)
FCM = C.get_chassis_tire_forces(FM, KAM, betaM)
MzM = C.calc_yaw_moment(FCM)
C.print_force_info(AyM, betaM, deltaM, MzM, KAM, SAM, SAP, loadM, FM, FEM, FCM)
# The main take away is that the tires aren't operating anywhere near peak slip angles
# The difference in front and rear lateral force is due to the effective torque from SA drag

In [ ]:
# Lets try manually messing with the angles
params = params
params['ack'] = -0.0
params['K_f'] = 16.5e3
params['K_r'] = 32e3
C = car.Car(params)
betaM = -2.5
deltaM = 11.6
betaM *= 1/con
deltaM *= 1/con
AyM, FxM, loadM, KAM, SAM, FM, FEM = C.Ay_Fx_from_angle(r, betaM, deltaM, extra=True)
SAP = C.calc_peak_slip_angles(loadM)
FCM = C.get_chassis_tire_forces(FM, KAM, betaM)
MzM = C.calc_yaw_moment(FCM)
C.print_force_info(AyM, betaM, deltaM, MzM, KAM, SAM, SAP, loadM, FM, FEM, FCM)
# The rear slip angles are low because the front of the car is grip limited
# Changing the roll stiffnesses helps a little bit and ups the cornering accel to 1.5g

In [ ]:
N = 50
r = 8
beta = np.linspace(-2.5, 8, N)
beta *= 1/con
delta = np.zeros(N, dtype='double')
Ay = np.zeros(N, dtype='double')
Fx = np.zeros(N, dtype='double')
load = np.zeros(N, dtype=[('fr', 'double'), ('fl', 'double'), ('rr', 'double'), ('rl', 'double')])
KA = np.zeros(N, dtype=[('fr', 'double'), ('fl', 'double'), ('rr', 'double'), ('rl', 'double')])
SA = np.zeros(N, dtype=[('fr', 'double'), ('fl', 'double'), ('rr', 'double'), ('rl', 'double')])
F = {}
FE = {}
for i in range(N):
    delta[i] = C.get_delta(r, beta[i])
    Ay[i], Fx[i], load[i], KA[i], SA[i], F[i], FE[i] = C.Ay_Fx_from_angle(r, beta[i], delta[i], extra=True)

sel = Ay > 0
plt.plot(beta[sel]*con, delta[sel]*con, 'tab:blue')
plt.title('Lateral acceleration and steer angle')
plt.xlabel(r'$\beta$ (deg)')
plt.ylabel(r'$\delta$ (deg)')
plt.twinx()
plt.plot(beta[sel]*con, Ay[sel], 'tab:red')
plt.ylabel(r'Ay (g)')
plt.legend([r'Ay'])
plt.show()

In [ ]:
# Lets try manually messing with the angles
params = params
params['a_s'] = params['l']*0.60
params['K_f'] = 16.5e3
params['K_r'] = 32e3
C = car.Car(params)
deltaM = 9.5
betaM = deltaM-14
betaM *= 1/con
deltaM *= 1/con
AyM, FxM, loadM, KAM, SAM, FM, FEM = C.Ay_Fx_from_angle(r, betaM, deltaM, extra=True)
SAP = C.calc_peak_slip_angles(loadM)
FCM = C.get_chassis_tire_forces(FM, KAM, betaM)
MzM = C.calc_yaw_moment(FCM)
C.print_force_info(AyM, betaM, deltaM, MzM, KAM, SAM, SAP, loadM, FM, FEM, FCM)
# The rear slip angles are low because the front of the car is grip limited
# Changing the roll stiffnesses helps a little bit and ups the cornering accel to 1.5g

In [ ]:
params = {
    'W_uf' : 111,    #Front unsprung weight (N)
    'W_ur' : 111,    #Rear unsprung weight  (N)
    'W_s'  : 2446,   #Sprung weight         (N)
    # CG locations
    'z_wf' : 0.21,   #Front unsprung height (m)
    'z_wr' : 0.21,   #Rear unsprung height  (m)
    'h_s'  : 0.23,   #Sprung height         (m)
    'a_s'  : 0.76,   #Sprung distance from front axle (m)
    # Car geometry
    'l'    : 1.524,  #Wheelbase   (m)
    't_f'  : 1.22,   #Front track (m)
    't_r'  : 1.22,   #Rear track  (m)
    # Roll center heights
    'z_rf' : 0.038,  #Front roll center height (m)
    'z_rr' : 0.076,  #Rear roll center height  (m)
    # Roll rates
    'K_f'  : 16.5e3, #Front roll rate (N-m/rad)
    'K_r'  : 32e3, #Rear roll rate  (N-m/rad)
    # Steering
    'ack'  : -0.0,    #Steering Ackerman (%)
    'toe_f': 0.5,   #Front toe (deg)
    'toe_r': 1.5,    #Rear toe  (deg)
    'tire' : tire.Hoosier10X7()
}
C = car.Car(params)
N = 50
r = 8
beta = np.linspace(-2.5, 8, N)
beta *= 1/con
delta = np.zeros(N, dtype='double')
Ay = np.zeros(N, dtype='double')
Fx = np.zeros(N, dtype='double')
load = np.zeros(N, dtype=[('fr', 'double'), ('fl', 'double'), ('rr', 'double'), ('rl', 'double')])
KA = np.zeros(N, dtype=[('fr', 'double'), ('fl', 'double'), ('rr', 'double'), ('rl', 'double')])
SA = np.zeros(N, dtype=[('fr', 'double'), ('fl', 'double'), ('rr', 'double'), ('rl', 'double')])
F = {}
FE = {}
for i in range(N):
    delta[i] = C.get_delta(r, beta[i])
    Ay[i], Fx[i], load[i], KA[i], SA[i], F[i], FE[i] = C.Ay_Fx_from_angle(r, beta[i], delta[i], extra=True)

sel = Ay > 0
plt.plot(beta[sel]*con, delta[sel]*con, 'tab:blue')
plt.title('Lateral acceleration and steer angle')
plt.xlabel(r'$\beta$ (deg)')
plt.ylabel(r'$\delta$ (deg)')
plt.twinx()
plt.plot(beta[sel]*con, Ay[sel], 'tab:red')
plt.ylabel(r'Ay (g)')
plt.legend([r'Ay'])
plt.show()